In [45]:
import logging
import threading
import time

In [46]:
def thread_function(name, secondary_id):
    logging.info(f'Thread %s:%s is starting' % (name, secondary_id))
    time.sleep(2)
    logging.info(f'Thread %s:%s is ending' % (name, secondary_id))

In [47]:
format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")

In [58]:
def parent_method():
    logging.info("Main    : before creating thread")
#     x = threading.Thread(target=thread_function, args=('one', 1), daemon=True)
    x = threading.Thread(target=thread_function, args=('one', 1))
    logging.info("Main    : before running thread")
    x.start()
    logging.info("Main    : wait for the thread to finish")
#     x.join()
    logging.info("Main    : all done")
    
# Note that daemon setting won't work in ipython. If I had used
# __name__ == '__main__' and then invoked the script, the use of daemon setting can be demo'ed

In [60]:
parent_method()

18:35:14: Main    : before creating thread
18:35:14: Main    : before running thread
18:35:14: Thread one:1 is starting
18:35:14: Main    : wait for the thread to finish
18:35:16: Thread one:1 is ending
18:35:16: Main    : all done
